### Self Training
- self-Training is a semi supervised learning approach that leverages a small labeled dataset alongside a larger unlabeled dataset. the model is initially trained on on labeled data. the confident prediction (those with high certainty ) are then added too the labeled dataset, and the process is repeated to imporve the model  

In [1]:
from sklearn.ensemble import RandomForestClassifier # this is used to train the model for classification and prediction
from sklearn.datasets import make_classification # this is used to create a synthetic dataset for testing 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
x,y = make_classification(n_samples=200, n_features=5, random_state=42)
x_labeled, x_unlabeled , y_labeled , _ = train_test_split(x,y, test_size=0.7, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(x_labeled, y_labeled)

# Initializing the model 
for _ in range(5):
    #preict probabilities on the unlabeled data 
    probs = model.predict_proba(x_unlabeled)
    high_confidence_idx = np.where(np.max(probs,axis=1)> 0.9)[0]
    
    #adding high confidence predictions to the labeled data 
    x_labeled= np.vstack([x_labeled,x_unlabeled[high_confidence_idx]])
    y_labeled = np.hstack([y_labeled, np.argmax(probs[high_confidence_idx], axis=1)])
    
    #remove confident samples from th e unlabeled dataset
    x_unlabeled = np.delete(x_unlabeled, high_confidence_idx, axis=0)
    
    #retrain the model
    model.fit(x_labeled, y_labeled)
    
# Evaluate the model
x_train, x_test, y_train, y_test = train_test_split(x_labeled, y_labeled, test_size=0.3, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracvy ", accuracy)    

Accuracvy  1.0
